# Visualizing sensor activities

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import numpy as np
from tqdm import tqdm

In [ ]:
src_dir = Path("/work/neuro/epochs")
sub_files = list(src_dir.glob('*'))
print(f"Found {len(sub_files)} subject files: {sub_files}")

statistics = {}

for i, file in enumerate(tqdm(sub_files, desc="Aggregating evoked statistics")):
    print(f"\nProcessing file {i}: {file}")
    base = joblib.load(file)

    subject_id = base['subject_id']
    epochs = base['epochs']        # (n_epochs, n_sensors, n_times)
    print(f"Epochs shape: {epochs.shape}")
    y = np.array(base['y'])        # (n_epochs,)

    classes = np.unique(y)
    results = {}

    for c in classes:
        mask = (y == c)
        epochs_c = epochs[mask]  # (n_c, n_sensors, n_times)

        if epochs_c.shape[0] == 0:
            print(f"Skipping class {c}: no epochs.")
            continue

        # Evoke across epochs (mean over epochs)
        evoked_c = epochs_c.mean(axis=0)  # (n_sensors, n_times)

        # Compute SEM across epochs for each sensor
        sem_c = epochs_c.std(axis=0, ddof=1) / np.sqrt(epochs_c.shape[0])  # (n_sensors, n_times)

        evoked_mean = evoked_c.mean(axis=0) 
        sem_mean = sem_c.mean(axis=0)

        results[c] = {
            "evoked": evoked_c,          # full sensor × time
            "sem": sem_c,                # full sensor × time
            "evoked_mean": evoked_mean,  # mean over sensors
            "sem_mean": sem_mean,
            "n_epochs": epochs_c.shape[0],
        }

    statistics[subject_id] = results

print("\n✅ Aggregation complete.")

plotting the evoked averaged signal for all sensors

In [ ]:
# Assuming same time vector for all subjects (adjust if different)
times = np.linspace(-0.2, 1.0, 1201)  # tmin=-0.2, tmax=1.0, sfreq=1000Hz → 1201 samples

# --- Aggregate group-level statistics ---
class_data = {}

for subject_id, subject_data in statistics.items():
    for class_id, class_stats in subject_data.items():
        if class_id not in class_data:
            class_data[class_id] = []
        class_data[class_id].append(class_stats["evoked_mean"])  # shape: (n_times,)

# --- Compute group mean and SEM across subjects ---
group_results = {}
for class_id, subject_means in class_data.items():
    subj_array = np.stack(subject_means, axis=0)  # (n_subjects, n_times)
    group_mean = subj_array.mean(axis=0)
    group_sem = subj_array.std(axis=0, ddof=1) / np.sqrt(subj_array.shape[0])
    group_results[class_id] = {
        "mean": group_mean,
        "sem": group_sem,
        "n_subjects": subj_array.shape[0],
    }

# --- Plotting ---
plt.figure(figsize=(10, 5))

n_classes = len(group_results)
color_map = plt.cm.Blues  # a balanced colormap
class_colors = color_map(np.linspace(0.4, 0.99, n_classes))



labels = ["NE - 1", "WG - 2", "ACE - 3", "CE - 4"]

for i, class_id in enumerate(sorted(group_results)):
    mean_c = group_results[class_id]["mean"]
    sem_c = group_results[class_id]["sem"]
    plt.plot(times, mean_c, color=class_colors[i], label=labels[i], linewidth=2)
    plt.fill_between(times, mean_c - sem_c, mean_c + sem_c, color=class_colors[i], alpha=0.15)

# Add horizontal and vertical reference lines
plt.axhline(0, color='k', linestyle='--', linewidth=0.8)  # horizontal at y=0
plt.axvline(0, color='k', linestyle='--', linewidth=0.8)  # vertical at x=0

# Set x-axis ticks every 100 ms
tick_interval = 0.1  # seconds
xticks = np.arange(times[0], times[-1]+tick_interval, tick_interval)
plt.xticks(xticks)

plt.xlabel('Time (s)')
plt.ylabel('Magnetic field (fT)')
plt.title('Group-Level Evoked Response (MAG Sensors mean)')
plt.legend(loc='upper left')

# Minimal aesthetic
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig("plots/group_average_all_sensors.png", dpi=300)
plt.show()
